1. **Корутина для подсчета уникальных слов (0.5б)**  
   Напишите корутину `unique_word_counter`, которая принимает строки (предложения), разбивает их на слова и подсчитывает уникальные слова. При отправке `None` корутина завершает выполнение и печатает итоговое количество уникальных слов.  
   **Пример:**  
   ```python
   coro = unique_word_counter()
   next(coro)
   coro.send("hello world")
   coro.send("hello again")
   coro.send(None)  # -> должно печатать 3, так как слова "hello", "world" и "again" уникальны. Это должно быть только число, подумайте как возвращать значение и обрабатывать результат вызова coro.send(None)
   ```

2. **Корутина с ограничением по времени выполнения (1б)**  
   Реализуйте корутину `timed_collector`, которая принимает целые числа и добавляет их к сумме. Если время выполнения корутины с момента ее запуска превышает заданное значение `time_limit` (в секундах), корутина завершается и возвращает текущую сумму. Считайте, что каждый yield значения из корутины занимает время, случайно распределенное между 1 и 3 секундами (реализуйте функцию для этого).
   **Пример использования:**  
   ```python
   coro = timed_collector(time_limit=5)
   next(coro)
   coro.send(10)
   coro.send(20)
   coro.send(30)
   coro.send(40)
   coro.send(50)
   # Через 5 секунд выполнения:
   # Завершение, возвращает текущую сумму: 30, 60 или 100. А может, и 150?
   # Запустите код несколько раз, убедитесь, что возвращаются разные числа!
   # Используйте корректную обработку строк формата coro.send(...)
   ```

3. **Корутина для динамической фильтрации данных с прерыванием (1б)**  
   Напишите корутину `dynamic_filter`, которая принимает числовые значения и фильтрует их по условию, переданному как лямбда-функция. Если отправить значение `None`, корутина завершает выполнение и возвращает список всех значений, прошедших фильтр. Если передать новое условие (новую лямбда-функцию), оно заменяет предыдущее.  
   **Пример использования:**  
   ```python
   coro = dynamic_filter(lambda x: x % 2 == 0)
   next(coro)
   coro.send(10)  # Проходит фильтр
   coro.send(15)  # Не проходит фильтр
   coro.send(lambda x: x > 10)  # Изменение условия фильтрации
   coro.send(15)  # Проходит новый фильтр
   coro.send(None)  # Завершение, возвращает [10, 15]
   ```

4. **Циклический генератор с обработкой исключений для отдельных заданий (0.5б)**  
   Создайте очередь заданий с помощью `deque`, где каждое задание — генератор, возвращающий числа. Реализуйте корутину `cycle_task_queue`, которая поочередно обрабатывает задания, пока очередь не опустеет. Если задание выбрасывает `StopIteration`, оно должно удаляться из очереди. Если задание выбрасывает `ValueError`, корутина должна просто пропустить это задание и перейти к следующему.  
   **Пример использования:**  
   ```python
   from collections import deque

   def task_1():
       yield 1
       yield 2
       raise ValueError("Task 1 error")

   def task_2():
       yield 3
       yield 4
       yield 5

   tasks = deque([task_1(), task_2()])
   coro = cycle_task_queue(tasks)
   for item in coro:
       print(item)
   ```

5. **Объединение данных из нескольких файлов с фильтром по содержимому, или немного искусственный пример на yield from (2б)**  
Вам даны три генератора (`file_reader_txt`, `file_reader_csv`, `file_reader_json`), каждый из которых читает данные из файлов различных форматов: текстового (`TXT`), CSV и JSON. Эти генераторы возвращают строки из своих соответствующих файлов. Ваша задача — реализовать две корутины: `merge_filtered_files` и `filter_and_write`.

- **`merge_filtered_files`** должна принимать:
   - Функцию фильтрации `filter_func`,
   - Имя выходного файла `output_filename`,
   - Набор генераторов, которые предоставляют строки из разных файлов.

   Корутина `merge_filtered_files` должна использовать `yield from`, чтобы объединить строки из каждого генератора в одном потоке, затем направлять их в корутину `filter_and_write` для фильтрации и записи в выходной файл.
   
- **`filter_and_write`** должна принимать:
   - Функцию фильтрации `filter_func`,
   - Объект файла для записи данных.
   
   `filter_and_write` будет ожидать строки от `merge_filtered_files`, фильтровать их по условию `filter_func` и записывать только те строки, которые удовлетворяют этому условию, в выходной файл.

**Реализуйте решение, в котором:**
1. `merge_filtered_files` поочередно получает строки из каждого переданного генератора, используя `yield from` (возможно, вам могут понадобиться генераторные выражения).
2. Каждая строка направляется в `filter_and_write`, который выполняет фильтрацию и запись.
3. В `filter_and_write` используется `yield`, чтобы реализовать взаимодействие с `yield from` в `merge_filtered_files`


```python
import csv
import json

# Генератор для чтения строк из текстового файла (TXT)
def file_reader_txt(filename):
    with open(filename, 'r') as file:
        for line in file:
            yield line.strip()

# Генератор для чтения строк из CSV-файла
def file_reader_csv(filename):
    with open(filename, 'r') as file:
        reader = csv.reader(file)
        for row in reader:
            yield ', '.join(row)  # Преобразуем каждую строку CSV в строку текста

# Генератор для чтения строк из JSON-файла
def file_reader_json(filename):
    with open(filename, 'r') as file:
        data = json.load(file)
        for item in data:
            yield json.dumps(item)  # Преобразуем каждый элемент JSON в строку

# Корутина для фильтрации и записи данных в файл
def filter_and_write(filter_func, output_file):
    # Принимает строки от merge_filtered_files через send
    # Применяет filter_func к каждой строке
    # Если строка удовлетворяет условию, записывает её в output_file
    # Использует yield для поддержания взаимодействия через yield from
    pass

# Корутина для объединения данных из нескольких генераторов с фильтрацией
def merge_filtered_files(filter_func, output_filename, *generators):
    # Открывает output_filename для записи
    # Инициализирует filter_and_write для фильтрации и записи строк
    # Использует yield from для поочередного получения строк от каждого генератора
    # и их передачи в filter_and_write через send
    # может быть что-то такое: yield from (writer.send(line) for line in generator), или явный цикл
    pass

# Функция фильтрации: проверяет, содержит ли строка слово "keyword"
def contains_keyword(line):
    return "keyword" in line

# Инициализация корутины и передача генераторов для объединения
merge = merge_filtered_files(
    contains_keyword,
    'merged_output.txt',
    file_reader_txt('file1.txt'),
    file_reader_csv('file2.csv'),
    file_reader_json('file3.json')
)

# Запуск корутины
for _ in merge:
    pass
```

В результате выполнения этого кода в выходном файле `merged_output.txt` должны оказаться только те строки из всех трёх файлов (TXT, CSV, JSON), которые содержат слово `"keyword"`.

Вот примеры файлов `file1.txt`, `file2.csv`, и `file3.json`, которые можно использовать для тестирования:

### Файл 1: `file1.txt`

Содержимое текстового файла:
```
This is a test line with keyword.
Another line without the keyword.
This keyword is very important.
A simple line without the important word.
```

### Файл 2: `file2.csv`

Содержимое CSV-файла:
```
name,description
Alice,This description contains the keyword somewhere.
Bob,Nothing special here.
Carol,Another entry with keyword here.
Dave,Just some random text without it.
```

### Файл 3: `file3.json`

Содержимое JSON-файла:
```json
[
    {"id": 1, "text": "The keyword appears in this JSON object."},
    {"id": 2, "text": "This one does not have it."},
    {"id": 3, "text": "Another keyword-rich JSON line."},
    {"id": 4, "text": "Yet another line without the keyword."}
]
```

### Ожидаемый результат

После выполнения программы, файл `merged_output.txt` должен содержать только строки, в которых присутствует слово `"keyword"`:

**merged_output.txt**
```
This is a test line with keyword.
Another line without the keyword.
This keyword is very important.
Alice, This description contains the keyword somewhere.
Carol, Another entry with keyword here.
{"id": 1, "text": "The keyword appears in this JSON object."}
{"id": 3, "text": "Another keyword-rich JSON line."}
{"id": 4, "text": "Yet another line without the keyword."}
```

### Код для создания этих файлов:

```python
with open('file1.txt', 'w') as f:
    f.write("""This is a test line with keyword.
Another line without the keyword.
This keyword is very important.
A simple line without the important word.""")

with open('file2.csv', 'w') as f:
    f.write("""name,description
Alice,This description contains the keyword somewhere.
Bob,Nothing special here.
Carol,Another entry with keyword here.
Dave,Just some random text without it.""")

import json
data = [
    {"id": 1, "text": "The keyword appears in this JSON object."},
    {"id": 2, "text": "This one does not have it."},
    {"id": 3, "text": "Another keyword-rich JSON line."},
    {"id": 4, "text": "Yet another line without the keyword."}
]
with open('file3.json', 'w') as f:
    json.dump(data, f)
```

6. **Рекурсивное объединение и фильтрация вложенных данных (3б)**  

У вас есть вложенные структуры данных, состоящие из списков и словарей, которые могут содержать строки, а могут — другие списки и словари. Ваша цель — рекурсивно пройти по всей этой структуре и получить только те строки, которые соответствуют заданной функции фильтрации `filter_func`.

Реализуйте функцию `recursive_extract` — генератор, который рекурсивно обходит вложенные списки и словари, используя `yield from` для делегирования обработки вложенных структур, и возвращает строки.

```python
# Функция фильтрации
def contains_keyword(s):
    return "keyword" in s

# Генератор для рекурсивного обхода и извлечения всех строк
def recursive_extract(data):
    # Проверяет тип data:
    # - Если data — строка, возвращает её
    # - Если data — словарь, рекурсивно обходит все значения словаря
    # - Если data — список, рекурсивно обходит все элементы списка
    # Использует yield from для делегирования обработки вложенных структур
    pass

# Функция для фильтрации строк с использованием yield from
def filtered_strings(data, filter_func):
    for string in recursive_extract(data):
        if filter_func(string):
            yield string

# Вложенная структура данных
nested_data = {
    "messages": [
        "This is a message with keyword.",
        "Just a simple message.",
        {"comments": ["Another keyword here.", "Nothing here."]},
        ["List with keyword.", "Another list entry."]
    ],
    "notes": "This is a single note without the word.",
    "logs": {
        "entries": [
            "Log entry with keyword.",
            {"subentry": "A nested keyword here."}
        ]
    }
}

# Итерируемся по отфильтрованным строкам
for line in filtered_strings(nested_data, contains_keyword):
    print(line)
```

### Ожидаемый вывод

```
This is a message with keyword.
Another keyword here.
List with keyword.
Log entry with keyword.
A nested keyword here.
```

In [71]:
#1
def unique_word_counter():
    unique_words = set()

    while True:
        sentence = yield
        if sentence is None:
          break
        for word in sentence.split(' '):
            unique_words.add(word)

    return len(unique_words)



coro = unique_word_counter()
next(coro)
coro.send("hello world")
coro.send("hello again")

try:
    coro.send(None)
except StopIteration as e:
    print(e.value)

# -> должно печатать 3, так как слова "hello", "world" и "again" уникальны. Это должно быть только число, подумайте как возвращать значение и обрабатывать результат вызова coro.send(None)

3


In [105]:
#2

# Корутина с ограничением по времени выполнения (1б)
# Реализуйте корутину timed_collector, которая принимает целые числа и добавляет их к сумме.
# Если время выполнения корутины с момента ее запуска превышает заданное значение time_limit (в секундах),
# корутина завершается и возвращает текущую сумму. Считайте, что каждый yield значения из корутины занимает время,
# случайно распределенное между 1 и 3 секундами (реализуйте функцию для этого). Пример использования:

import time
import random


def random_time():
  return random.uniform(1, 3)


def timed_collector(time_limit):
  start = time.time()
  sum = 0
  while True:
    number = yield time.sleep(random_time())
    sum+= number
    end = time.time()
    if end - start > time_limit:
       break

  return sum


coro = timed_collector(time_limit=5)
next(coro)
try:
  coro.send(10)
  coro.send(20)
  coro.send(30)
  coro.send(40)
  coro.send(50)
except StopIteration as e:
  print(e.value)

# Через 5 секунд выполнения:
# Завершение, возвращает текущую сумму: 30, 60 или 100. А может, и 150?
# Запустите код несколько раз, убедитесь, что возвращаются разные числа!
# Используйте корректную обработку строк формата coro.send(...)

100


In [110]:
coro = timed_collector(time_limit=5)
next(coro)


try:
  coro.send(10)
  coro.send(20)
  coro.send(30)
  coro.send(40)
  coro.send(50)
except StopIteration as e:
  print(e.value)

60


In [145]:
#3
# Корутина для динамической фильтрации данных с прерыванием (1б)
# Напишите корутину dynamic_filter, которая принимает числовые значения и фильтрует их по условию, переданному как лямбда-функция.
# Если отправить значение None, корутина завершает выполнение и возвращает список всех значений, прошедших фильтр.
# Если передать новое условие (новую лямбда-функцию), оно заменяет предыдущее.

def dynamic_filter(func=None):
    res=[]
    while True:
      input = yield
      if type(input) == int:
        if input is None:
          break
        if func(input) == True:
          res.append(input)
          print(res)
      else:
        func = input

    return res



coro = dynamic_filter(lambda x: x % 2 == 0)
next(coro)
coro.send(10)  # Проходит фильтр
coro.send(15)  # Не проходит фильтр
coro.send(lambda x: x > 10)  # Изменение условия фильтрации
coro.send(15)  # Проходит новый фильтр
coro.send(None)  # Завершение, возвращает [10, 15]


[10]
[10, 15]


In [147]:
#4

# Циклический генератор с обработкой исключений для отдельных заданий (0.5б)
# Создайте очередь заданий с помощью deque, где каждое задание — генератор, возвращающий числа.
# Реализуйте корутину cycle_task_queue, которая поочередно обрабатывает задания, пока очередь не опустеет.
# Если задание выбрасывает StopIteration, оно должно удаляться из очереди.
# Если задание выбрасывает ValueError, корутина должна просто пропустить это задание и перейти к следующему.


from collections import deque

def task_1():
    yield 1
    yield 2
    raise ValueError("Task 1 error")

def task_2():
    yield 3
    yield 4
    yield 5


def cycle_task_queue(tasks):
  while tasks:
      gen = tasks.popleft()
      try:
          value = next(gen)
          yield value
          tasks.append(gen)
      except StopIteration:
          tasks.remove(gen)
      except ValueError:
        continue



tasks = deque([task_1(), task_2()])
coro = cycle_task_queue(tasks)
for item in coro:
    print(item)

1
3
2
4
5


ValueError: <generator object task_2 at 0x782bd1c13680> is not in deque

In [1]:
#5
with open('file1.txt', 'w') as f:
    f.write("""This is a test line with keyword.
Another line without the keyword.
This keyword is very important.
A simple line without the important word.""")

with open('file2.csv', 'w') as f:
    f.write("""name,description
Alice,This description contains the keyword somewhere.
Bob,Nothing special here.
Carol,Another entry with keyword here.
Dave,Just some random text without it.""")

import json
data = [
    {"id": 1, "text": "The keyword appears in this JSON object."},
    {"id": 2, "text": "This one does not have it."},
    {"id": 3, "text": "Another keyword-rich JSON line."},
    {"id": 4, "text": "Yet another line without the keyword."}
]
with open('file3.json', 'w') as f:
    json.dump(data, f)

In [21]:
#5
import csv
import json

# Генератор для чтения строк из текстового файла (TXT)
def file_reader_txt(filename):
    with open(filename, 'r') as file:
        for line in file:
            yield line.strip()

# Генератор для чтения строк из CSV-файла
def file_reader_csv(filename):
    with open(filename, 'r') as file:
        reader = csv.reader(file)
        for row in reader:
            yield ', '.join(row)  # Преобразуем каждую строку CSV в строку текста

# Генератор для чтения строк из JSON-файла
def file_reader_json(filename):
    with open(filename, 'r') as file:
        data = json.load(file)
        for item in data:
            yield json.dumps(item)  # Преобразуем каждый элемент JSON в строку

# Корутина для фильтрации и записи данных в файл
def filter_and_write(filter_func, output_file):
  while True:
    row = yield
    if row is None:
      break
    if filter_func(row):
      with open (output_file, 'a') as f:
        f.write(row + '\n')

    # Принимает строки от merge_filtered_files через send
    # Применяет filter_func к каждой строке
    # Если строка удовлетворяет условию, записывает её в output_file
    # Использует yield для поддержания взаимодействия через yield from


# Корутина для объединения данных из нескольких генераторов с фильтрацией
def merge_filtered_files(filter_func, output_filename, *generators):
    with open (output_filename, 'w') as f:
      writer = filter_and_write(filter_func, output_filename)
      next(writer)

      def generator():
        for generator in generators:
          yield from generator

      gen = generator()
      for line in gen:
        if line != None:
          writer.send(line)




    # Открывает output_filename для записи
    # Инициализирует filter_and_write для фильтрации и записи строк
    # Использует yield from для поочередного получения строк от каждого генератора
    # и их передачи в filter_and_write через send
    # может быть что-то такое: yield from (writer.send(line) for line in generator), или явный цикл


# Функция фильтрации: проверяет, содержит ли строка слово "keyword"
def contains_keyword(line):
    return "keyword" in line

# Инициализация корутины и передача генераторов для объединения
merge = merge_filtered_files(
    contains_keyword,
    'merged_output.txt',
    file_reader_txt('file1.txt'),
    file_reader_csv('file2.csv'),
    file_reader_json('file3.json')
)

# Запуск корутины
for _ in merge:
    print(_)

TypeError: 'NoneType' object is not iterable

In [25]:
with open ('merged_output.txt', 'r') as f:
  print(*f)

This is a test line with keyword.
 Another line without the keyword.
 This keyword is very important.
 Alice, This description contains the keyword somewhere.
 Carol, Another entry with keyword here.
 {"id": 1, "text": "The keyword appears in this JSON object."}
 {"id": 3, "text": "Another keyword-rich JSON line."}
 {"id": 4, "text": "Yet another line without the keyword."}



In [ ]:
#6 - рекурсию не понимаю(((